In [1]:
import os
#!pip install findspark pyspark==3.4.0
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
#!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!wget -q https://archive.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.2.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.2.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3.2"

# Start a SparkSession
import findspark
findspark.init()


Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,325 kB]
Get:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease [23.8 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,671 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()


In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
from pyspark.sql import functions as F

url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv("file://" + SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)


In [4]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView("home_sales")
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- date_built: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: integer (nullable = true)
 |-- waterfront: integer (nullable = true)
 |-- view: integer (nullable = true)



In [5]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
result = spark.sql("""
    SELECT YEAR(date) AS SaleYear, ROUND(AVG(price), 2) AS AvgPrice
    FROM home_sales
    WHERE bedrooms = 4
    GROUP BY SaleYear
    ORDER BY SaleYear
""")

# Show the result
display(result.toPandas())


,SaleYear,AvgPrice
0,2019,300263.70
1,2020,298353.78
2,2021,301819.44
3,2022,296363.88


In [6]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
result_3bed_3bath = spark.sql("""
    SELECT date_built AS YearBuilt, ROUND(AVG(price), 2) AS AvgPrice
    FROM home_sales
    WHERE bedrooms = 3 AND bathrooms = 3
    GROUP BY YearBuilt
    ORDER BY YearBuilt
""")

# Show the result
display(result_3bed_3bath.toPandas())

,YearBuilt,AvgPrice
0,2010,292859.62
1,2011,291117.47
2,2012,293683.19
3,2013,295962.27
4,2014,290852.27
5,2015,288770.30
6,2016,290555.07
7,2017,292676.79


In [7]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
result_filtered = spark.sql("""
    SELECT date_built AS YearBuilt, ROUND(AVG(price), 2) AS AvgPrice
    FROM home_sales
    WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
    GROUP BY YearBuilt
    ORDER BY YearBuilt
""")

# Show the result
display(result_filtered.toPandas())


,YearBuilt,AvgPrice
0,2010,285010.22
1,2011,276553.81
2,2012,307539.97
3,2013,303676.79
4,2014,298264.72
5,2015,297609.97
6,2016,293965.10
7,2017,280317.58


In [8]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.
start_time = time.time()

result_avg_price_view = spark.sql("""
    SELECT view, ROUND(AVG(price), 2) AS AvgPrice
    FROM home_sales
    WHERE price >= 350000
    GROUP BY view
    ORDER BY view
""")

# Show the result
display(result_avg_price_view.toPandas())

print("--- %s seconds ---" % (time.time() - start_time))

,view,AvgPrice
0,0,403848.51
1,1,401044.25
2,2,397389.25
3,3,398867.60
4,4,399631.89
...,...,...
96,96,1017815.92
97,97,1129040.15
98,98,1053739.33
99,99,1061201.42


--- 4.646274566650391 seconds ---


In [9]:
# 7. Cache the the temporary table home_sales.
df.createOrReplaceTempView("home_sales")
df.cache()

DataFrame[id: string, date: string, date_built: int, price: int, bedrooms: int, bathrooms: int, sqft_living: int, sqft_lot: int, floors: int, waterfront: int, view: int]

In [10]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [11]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time_cached = time.time()

result_avg_price_view_cached = spark.sql("""
    SELECT view, ROUND(AVG(price), 2) AS AvgPrice
    FROM home_sales
    WHERE price >= 350000
    GROUP BY view
    ORDER BY view
""")

# Show the result
display(result_avg_price_view_cached.toPandas())

print("--- Cached Runtime: %s seconds ---" % (time.time() - start_time_cached))


# Run the same query on uncached data
start_time_uncached = time.time()

result_avg_price_view_uncached = spark.sql("""
    SELECT view, ROUND(AVG(price), 2) AS AvgPrice
    FROM home_sales
    WHERE price >= 350000
    GROUP BY view
    ORDER BY view
""")

# Show the result
display(result_avg_price_view_uncached.toPandas())

print("--- Uncached Runtime: %s seconds ---" % (time.time() - start_time_uncached))

,view,AvgPrice
0,0,403848.51
1,1,401044.25
2,2,397389.25
3,3,398867.60
4,4,399631.89
...,...,...
96,96,1017815.92
97,97,1129040.15
98,98,1053739.33
99,99,1061201.42


--- Cached Runtime: 5.369330644607544 seconds ---


,view,AvgPrice
0,0,403848.51
1,1,401044.25
2,2,397389.25
3,3,398867.60
4,4,399631.89
...,...,...
96,96,1017815.92
97,97,1129040.15
98,98,1053739.33
99,99,1061201.42


--- Uncached Runtime: 4.815003871917725 seconds ---


In [12]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
output_path = "/content/formatted_home_sales_partitioned.parquet"
df.write.mode("overwrite").partitionBy("date_built").parquet(output_path)

In [13]:
# 11. Read the parquet formatted data.
read_df = spark.read.parquet(output_path)
read_df.printSchema()
read_df.show()

root
 |-- id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- price: integer (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: integer (nullable = true)
 |-- waterfront: integer (nullable = true)
 |-- view: integer (nullable = true)
 |-- date_built: integer (nullable = true)

+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         

In [14]:
# 12. Create a temporary table for the parquet data.
read_df.createOrReplaceTempView("parquet_home_sales")

In [15]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.
start_time_parquet = time.time()

result_avg_price_view_parquet = spark.sql("""
    SELECT view, ROUND(AVG(price), 2) AS AvgPrice
    FROM parquet_home_sales
    WHERE price >= 350000
    GROUP BY view
    ORDER BY view
""")

# Show the result
display(result_avg_price_view_parquet.toPandas())

print("--- Parquet Runtime: %s seconds ---" % (time.time() - start_time_parquet))


# Compare with the cached version
start_time_cached = time.time()

result_avg_price_view_cached = spark.sql("""
    SELECT view, ROUND(AVG(price), 2) AS AvgPrice
    FROM home_sales
    WHERE price >= 350000
    GROUP BY view
    ORDER BY view
""")

# Show the result
display(result_avg_price_view_cached.toPandas())

print("--- Cached Runtime: %s seconds ---" % (time.time() - start_time_cached))

,view,AvgPrice
0,0,403848.51
1,1,401044.25
2,2,397389.25
3,3,398867.60
4,4,399631.89
...,...,...
96,96,1017815.92
97,97,1129040.15
98,98,1053739.33
99,99,1061201.42


--- Parquet Runtime: 5.499908924102783 seconds ---


,view,AvgPrice
0,0,403848.51
1,1,401044.25
2,2,397389.25
3,3,398867.60
4,4,399631.89
...,...,...
96,96,1017815.92
97,97,1129040.15
98,98,1053739.33
99,99,1061201.42


--- Cached Runtime: 3.655214786529541 seconds ---


In [16]:
# 14. Uncache the home_sales temporary table.
df.unpersist()

DataFrame[id: string, date: string, date_built: int, price: int, bedrooms: int, bathrooms: int, sqft_living: int, sqft_lot: int, floors: int, waterfront: int, view: int]

In [17]:
# 15. Check if the home_sales is no longer cached
df.is_cached

False

In [ ]:
#code will return False if the home_sales DataFrame is no longer cached.